In [26]:
import json
from pathlib import Path
import shutil

import pandas as pd
import numpy as np

from bw2io import bw2setup
from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba

In [3]:
print(bd.__version__)

(4, 0, 'DEV37')


In [4]:
pet_hiot = pd.DataFrame([[1,-2],[0,1]], # experiment setting 1 prod to 0
             index=pd.MultiIndex.from_tuples([('DK','prod1'),('DK','prod2')]),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

pet_hiot = pet_hiot.astype(pd.SparseDtype("float",0))


B = pd.DataFrame([[1,3],[1,2],[0,1]],
             index=pd.Index(['co2_air','ch4_air','co2_accelerated_air']),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

B = B.astype(pd.SparseDtype("float",0))

fd = pd.DataFrame.from_dict(
    {
    ('DK','Household'):{('DK','prod1'):-11,('DK','prod2'):-3},
    ('DK','Government'):{('DK','prod1'):-8,('DK','prod2'):-4},
    ('DK','Capital'):{('DK','prod1'):-4,('DK','prod2'):-2}
        }
    )

Bfd = pd.DataFrame([[1,3],],
index=pd.Index(['co2_air',]),
columns = pd.MultiIndex.from_tuples([('DK','Household'),
('DK','Government')])).astype(pd.SparseDtype("float",0))

In [5]:
print(pet_hiot)
print(B)
print(fd)
print(Bfd)

           DK     
         act1 act2
DK prod1  1.0 -2.0
   prod2    0  1.0
                      DK     
                    act1 act2
co2_air              1.0  3.0
ch4_air              1.0  2.0
co2_accelerated_air    0  1.0
                DK                   
         Household Government Capital
DK prod1       -11         -8      -4
   prod2        -3         -4      -2
               DK           
        Household Government
co2_air       1.0        3.0


In [6]:
pfd = pd.DataFrame((np.eye(fd.shape[1])),index=fd.columns,columns=fd.columns)
fd_total = pd.concat([fd,pfd])
fd_total = fd_total.astype(pd.SparseDtype("float",0))

In [7]:
extended_hiot = pd.concat([pet_hiot,fd_total],axis=1).fillna(0)
extended_B = pd.concat([B,Bfd],axis=1).fillna(0)

In [8]:
print(extended_hiot)

                DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0


In [9]:
print(extended_B)

                      DK                          
                    act1 act2 Household Government
co2_air              1.0  3.0       1.0        3.0
ch4_air              1.0  2.0       0.0        0.0
co2_accelerated_air    0  1.0       0.0        0.0


In [10]:
path_to_intermediate = (Path.cwd()/'results')
path_to_intermediate.mkdir(exist_ok=True)

In [11]:
metadata_dict = {'prod1':{'unit':'kg','name':'product 1'},
                 'prod2':{'unit':'kg','name':'product 2'},
                 'Household':{'unit':'unit','name':'the household'},
                 'Government':{'unit':'unit','name':'the government'},
                 'Capital':{'unit':'unit','name':'capital investments'},

                 'co2_air':{'unit':'ton', # not standard units
                            'name':'carbon dioxide',
                            'compartment':('air',)},
                 'ch4_air':{'unit':'kg',
                            'name':'methane',
                            'compartment':('air',)},
                 'co2_accelerated_air':{'unit':'kg', # additional biosphere flow
                                        'name':'carbon dioxide accelerated',
                                        'compartment':('air',)}
                }

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)

In [12]:
tidy_tables(extended_hiot,extended_B,path_to_intermediate)

In [13]:
bd.projects.set_current('test_io_importer')

In [14]:
bw2setup()

Biosphere database already present!!! No setup is needed


In [15]:
try:
    del bd.databases['pet_io_db biosphere']
    del bd.databases['pet_io_db']
    bd.Database('pet_io_db').delete()
    bd.Database('pet_io_db biosphere').delete()
except KeyError:
    print('db not there')

c:\Users\Astudillo\repos20\brightway2-io\venv\Lib\site-packages\bw2data\backends\base.py:625: UserWarning: 
            Please use `del databases['pet_io_db']` instead.
            Otherwise, the metadata and database get out of sync.
            Call `.delete(warn=False)` to skip this message in the future.
            
  warnings.warn(MESSAGE.format(self.name), UserWarning)
c:\Users\Astudillo\repos20\brightway2-io\venv\Lib\site-packages\bw2data\backends\base.py:625: UserWarning: 
            Please use `del databases['pet_io_db biosphere']` instead.
            Otherwise, the metadata and database get out of sync.
            Call `.delete(warn=False)` to skip this message in the future.
            
  warnings.warn(MESSAGE.format(self.name), UserWarning)


In [16]:
pet_example = IOImporter(path_to_intermediate,'pet_io_db')
pet_example.apply_strategies()

In [17]:
pet_example.write_database()

100%|██████████| 1/1 [00:00<00:00, 1000.07it/s]


Vacuuming database 


100%|██████████| 5/5 [00:00<00:00, 165.93it/s]


Vacuuming database 
Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


In [27]:
test_db = bd.Database('pet_io_db')

In [28]:
act1 = test_db.get(code='act1|DK')
act2 = test_db.get(code='act2|DK')

In [20]:
ipcc_2021 = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [21]:
test_lca = bc.LCA({act2:1},ipcc_2021)

In [22]:
test_lca.lci()
test_lca.lcia()
test_lca.score

4655.12370300293

In [23]:
extended_hiot

DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0

In [24]:
test_lca.technosphere_matrix.todense()

matrix([[  1.,  -2., -11.,  -8.,  -4.],
        [  0.,   1.,  -3.,  -4.,  -2.],
        [  0.,   0.,   1.,   0.,   0.],
        [  0.,   0.,   0.,   1.,   0.],
        [  0.,   0.,   0.,   0.,   1.]])

In [36]:
ba.print_recursive_calculation(act2,ipcc_2021,use_matrix_values=True)

Fraction of score | Absolute score | Amount | Activity
0001 |  4655 |     1 | 'act2' (kilogram, DK, None)
  0.403 |  1874 |     2 | 'act1' (kilogram, DK, None)
